In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

In [7]:
data = load_dataset("valhalla/emoji-dataset")["train"]

In [16]:
count = 0
for row in data:
  if "man" in row["text"]: #all emojis with the term man or woman
    count += 1
count

610

In [18]:
def contains_man(row):
  return "man" in row["text"]

filtered = data.filter(contains_man)

Filter:   0%|          | 0/2749 [00:00<?, ? examples/s]

In [19]:
df = filtered.to_pandas()

In [21]:
!pip install pillow

In [25]:
import pandas as pd
import PIL

In [40]:
from PIL import Image, ImageOps
import numpy as np
import io
import random
from IPython.display import display

def process_image(bytes_arr):
  image = Image.open(io.BytesIO(bytes_arr))
  size = (64,64)
  resized_image = image.resize(size, Image.ANTIALIAS)
  image_array = np.array(resized_image)
  return image_array

def augment_image(og_array):
  og = Image.fromarray(og_array)
  augmentations = ["h", "v", "r"] #horizontal flip, vertical_flip, rotate
  augmentation = random.choice(augmentations)
  aug = None
  if augmentation == "h":
    aug = ImageOps.mirror(og)
  elif augmentation == "v":
    aug = ImageOps.flip(og)
  else:
    d = random.choice([90,180,270])
    aug = og.rotate(d)
  aug_array = np.array(aug)
  return aug_array

def show_image(image_array):
  image = Image.fromarray(image_array)
  display(image)


In [36]:
augmented_dataset = []
for image in df["image"]:
  bytes_arr = image["bytes"]
  image_arr = process_image(bytes_arr)
  augmented_dataset.append(image_arr)
  augmented_dataset.append(augment_image(image_arr))
final_dataset = np.array(augmented_dataset)





<ipython-input-35-cc9e2b5cb8e4>:9: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = image.resize(size, Image.ANTIALIAS)


In [51]:
np.save("processed_data.npy", final_dataset)